In [1]:
import duckdb

In [4]:
ARQUIVO_SOLARIMETRO = '../sonda-translator/Solarimetrica_2.parquet'
NOME_DA_TABELA = 'solarimetrica'

In [3]:
con = duckdb.connect(database=':memory:')

In [ ]:
# Cria as tabelas no banco de dados DuckDB a partir dos arquivos Parquet
con.execute(f"""CREATE TABLE IF NOT EXISTS {NOME_DA_TABELA} AS \
            SELECT * FROM read_parquet('{ARQUIVO_SOLARIMETRO}')""")

In [ ]:
# Exibe as primeiras linhas da tabela meteorológica
con.execute(f"SELECT * FROM {NOME_DA_TABELA} LIMIT 5").fetch_df()